In [ ]:
import asyncio
import logging
from collections import deque
from datetime import datetime, timedelta
import pandas as pd
import pyotp
from NorenRestApiPy.NorenApi import NorenApi
import numba_indicators
import nest_asyncio
import weakref
import numpy as np
import csv
import os

# Global variable to store the DataProcessor instance
global_data_processor = None
global_candle_end_finder = None
global_tick_collector = None
global_decision_maker = None

class TickCollector:
    def __init__(self, credentials_file="usercred.xlsx"):
        self.processing_lock = asyncio.Lock()
        self.api = None
        self.feed_opened = False
        self.ring_buffers = {}
        self.resampled_buffers = {}
        self.resampling_enabled = {}
        self.last_tick_time = {}
        self.active_subscriptions = set()
        self.RING_BUFFER_SIZE = 1000
        self.RING_BUFFER_RESAMPLE_SIZE = 1000
        self.VALID_TIMEFRAMES = ['5s','15s']
        
        self.logger = self._setup_logger()
        self._initialize_api(credentials_file)

    def _setup_logger(self):
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        return logger

    def _initialize_api(self, credentials_file):
        self.api = NorenApi(
            host="https://api.shoonya.com/NorenWClientTP/",
            websocket="wss://api.shoonya.com/NorenWSTP/"
        )
        credentials = pd.read_excel(credentials_file)
        user = credentials.iloc[0, 0]
        password = credentials.iloc[0, 1]
        vendor_code = credentials.iloc[0, 2]
        app_key = credentials.iloc[0, 3]
        imei = credentials.iloc[0, 4]
        qr_code = credentials.iloc[0, 5]
        factor2 = pyotp.TOTP(qr_code).now()

        self.api.login_result = self.api.login(
            userid=user,
            password=password,
            twoFA=factor2,
            vendor_code=vendor_code,
            api_secret=app_key,
            imei=imei
        )

    def create_ring_buffers(self, tokens):
        for token in tokens:
            if token not in self.ring_buffers:
                self.ring_buffers[token] = deque(maxlen=self.RING_BUFFER_SIZE)
                self.last_tick_time[token] = None
                self.logger.info(f"Created ring buffer for token: {token}")

    def create_resampled_buffers(self, tokens, timeframes):
        for token in tokens:
            if token not in self.resampled_buffers:
                self.resampled_buffers[token] = {}
                self.resampling_enabled[token] = {}
                self.logger.info(f"Created resampled buffers entry for token: {token}")
            
            for timeframe in timeframes:
                if timeframe not in self.resampled_buffers[token]:
                    self.resampled_buffers[token][timeframe] = deque(maxlen=self.RING_BUFFER_RESAMPLE_SIZE)
                    self.resampling_enabled[token][timeframe] = False
                    self.logger.info(f"Created resampled buffer for token {token} and timeframe: {timeframe}")

    async def set_resampling(self, token, timeframe, enable):
        if token in self.resampling_enabled and timeframe in self.resampling_enabled[token]:
            self.resampling_enabled[token][timeframe] = enable
            self.logger.info(f"Resampling {'enabled' if enable else 'disabled'} for token {token} and timeframe {timeframe}")
        else:
            self.logger.warning(f"Token {token} or timeframe {timeframe} not found in resampling_enabled")

    def event_handler_feed_update(self, tick_data):
        try:
            if 'lp' in tick_data and 'tk' in tick_data:
                timest = datetime.fromtimestamp(int(tick_data['ft'])).isoformat()
                token = tick_data['tk']
                if token in self.ring_buffers:
                    new_tick = {'tt': timest, 'ltp': float(tick_data['lp'])}
                    self.ring_buffers[token].append(new_tick)
                    self.last_tick_time[token] = datetime.fromisoformat(timest)
                else:
                    self.logger.warning(f"Token {token} not found in ring buffers. Ignoring tick.")
        except (KeyError, ValueError) as e:
            self.logger.error(f"Error processing tick data: {e}")

    async def connect_and_subscribe(self):
        retry_delay = 1
        max_retry_delay = 32
        max_retries = 10
        retries = 0
        while retries < max_retries:
            try:
                self.api.start_websocket(
                    order_update_callback=self.event_handler_order_update,
                    subscribe_callback=self.event_handler_feed_update,
                    socket_open_callback=self.open_callback,
                    socket_close_callback=self.close_callback
                )
                await self.wait_for_feed_open(timeout=30)
                self.logger.info("WebSocket connected successfully.")
                
                await self.manage_subscriptions('add', 'MCX|432294')
                #await self.manage_subscriptions('add', 'NSE|26009')
                # await self.manage_subscriptions('add', 'NSE|26000')
                
                retry_delay = 1
                retries = 0
                await self.monitor_connection()
            except Exception as e:
                self.logger.error(f"WebSocket connection error: {e}")
                retries += 1
                self.logger.info(f"Reconnecting in {retry_delay} seconds... (Attempt {retries}/{max_retries})")
                await asyncio.sleep(retry_delay)
                retry_delay = min(retry_delay * 2, max_retry_delay)

        if retries >= max_retries:
            self.logger.error("Max retries reached. Exiting.")
            raise Exception("Max retries reached")

    async def manage_subscriptions(self, command, subscription):
        token = subscription.split('|')[1]

        if command == 'add':
            if subscription not in self.active_subscriptions:
                self.api.subscribe([subscription])
                self.active_subscriptions.add(subscription)
                self.create_ring_buffers([token])
                self.create_resampled_buffers([token], self.VALID_TIMEFRAMES)
                self.logger.info(f"Subscribed to {subscription}")
                for timeframe in self.VALID_TIMEFRAMES:
                    await self.set_resampling(token, timeframe, True)
                self.logger.info(f"Resampling enabled for token {token} for all valid timeframes.")
            else:
                self.logger.warning(f"Already subscribed to {subscription}")
        elif command == 'remove':
            if subscription in self.active_subscriptions:
                self.api.unsubscribe([subscription])
                self.active_subscriptions.remove(subscription)
                self.logger.info(f"Unsubscribed from {subscription}")
            else:
                self.logger.warning(f"Not subscribed to {subscription}")

    async def wait_for_feed_open(self, timeout):
        start_time = asyncio.get_event_loop().time()
        while not self.feed_opened:
            if asyncio.get_event_loop().time() - start_time > timeout:
                raise TimeoutError("Timed out waiting for feed to open")
            await asyncio.sleep(1)

    async def monitor_connection(self):
        while True:
            if not self.feed_opened:
                self.logger.warning("Feed closed unexpectedly. Reconnecting...")
                raise Exception("Feed closed")
            await asyncio.sleep(5)

    def close_callback(self):
        self.feed_opened = False
        self.logger.warning("WebSocket connection closed.")
        self.logger.info("Attempting to reconnect...")

    def open_callback(self):
        if not self.feed_opened:
            self.feed_opened = True
            self.logger.info('Feed Opened')
        else:
            self.logger.warning('Feed Opened callback called multiple times.')

    def event_handler_order_update(self, data):
        self.logger.info(f"Order update: {data}")

    async def run(self):
        await asyncio.gather(
            self.connect_and_subscribe()            
        )
        
class DataProcessor:
    def __init__(self, tick_collector):
        self.tick_collector = tick_collector
        self.logger = logging.getLogger(__name__)
        self.last_processed_time = {}
        self.IDLE_THRESHOLD = timedelta(minutes=1)        

    async def run(self):
        while True:
            await asyncio.sleep(0.1)  # Reduced sleep time
            await self.process_data()

    async def process_data(self):
        try:
            tokens_to_process = []
            async with self.tick_collector.processing_lock:
                current_time = datetime.now()
                for token in self.tick_collector.ring_buffers.keys():
                    last_tick_time = self.tick_collector.last_tick_time.get(token)
                    if last_tick_time is None:
                        self.logger.info(f"No last tick time available for {token}")
                        continue
                    
                    time_since_last_tick = current_time - last_tick_time
                    if time_since_last_tick <= self.IDLE_THRESHOLD:
                        tokens_to_process.append(token)

            for token in tokens_to_process:
                for timeframe in self.tick_collector.VALID_TIMEFRAMES:
                    if self.tick_collector.resampling_enabled[token].get(timeframe, False):
                        await self.process_token_timeframe(token, timeframe)

        except Exception as e:
            self.logger.error(f"Error in process_data: {str(e)}")
            self.logger.exception("Detailed traceback:")

    async def process_token_timeframe(self, token, timeframe):
        try:
            ticks = None
            async with self.tick_collector.processing_lock:
                ticks = list(self.tick_collector.ring_buffers.get(token, deque()))
            
            if not ticks:
                return

            df_new = pd.DataFrame(ticks)
            df_new["tt"] = pd.to_datetime(df_new["tt"])
            df_new.set_index("tt", inplace=True)

            df_resampled = df_new['ltp'].resample(timeframe).ohlc()
            df_resampled = df_resampled.dropna(subset=['open', 'high', 'low', 'close'])

            # Indicator calculation
            try:
                supertrend, supertrend_direction = numba_indicators.supertrend_numba(
                    df_resampled['high'].values,
                    df_resampled['low'].values,
                    df_resampled['close'].values
                )
                df_resampled['supertrend'] = supertrend
                df_resampled['supertrend_direction'] = supertrend_direction

                jma, jma_direction = numba_indicators.jma_numba_direction(
                    df_resampled['close'].values
                )
                df_resampled['jma'] = jma
                df_resampled['jma_direction'] = jma_direction

                self.logger.debug(f"Indicator calculation completed for {token} at {timeframe}. Data shape: {df_resampled.shape}")
            except Exception as e:
                self.logger.error(f"Error calculating indicators for {token} at {timeframe}: {str(e)}")
                self.logger.exception("Detailed traceback:")
                return

            new_records = df_resampled.reset_index().to_dict('records')
            
            async with self.tick_collector.processing_lock:
                resampled_buffer = self.tick_collector.resampled_buffers[token].get(timeframe, deque(maxlen=self.tick_collector.RING_BUFFER_RESAMPLE_SIZE))
                resampled_buffer.clear()
                resampled_buffer.extend(new_records)
                self.tick_collector.resampled_buffers[token][timeframe] = resampled_buffer
                self.last_processed_time[(token, timeframe)] = df_resampled.index.max()

        except Exception as e:
            self.logger.error(f"Error processing {token} at {timeframe}: {str(e)}")
            self.logger.exception("Detailed traceback:")

    async def get_resampled_buffer_contents(self, token=None, timeframe=None):    
        if token is None and timeframe is None:
            return {t: {tf: list(b) for tf, b in buffers.items()} 
                    for t, buffers in self.tick_collector.resampled_buffers.items()}
        elif token is not None and timeframe is None:
            return {tf: list(b) for tf, b in self.tick_collector.resampled_buffers.get(token, {}).items()}
        elif token is not None and timeframe is not None:
            return list(self.tick_collector.resampled_buffers.get(token, {}).get(timeframe, deque()))
        else:  # token is None and timeframe is not None
            return {t: list(buffers.get(timeframe, deque())) 
                    for t, buffers in self.tick_collector.resampled_buffers.items()}
        
class CandleEndFinder:
    def __init__(self, data_processor: DataProcessor):
        self.data_processor = data_processor
        self.logger = logging.getLogger(__name__)
        self.completed_candles_dfs = {}
        self.last_processed_candle = {}
        self.new_candle_queue = asyncio.Queue()  # New queue for publishing candles
        self.IDLE_THRESHOLD = timedelta(minutes=1) 

    async def run(self):
        while True:
            try:
                await self.find_completed_candles()
                await asyncio.sleep(.5)  # Adjust as needed speed of candle detection
            except Exception as e:
                self.logger.error(f"Error in CandleEndFinder run loop: {e}")
                await asyncio.sleep(5)  # Wait a bit longer before retrying after an error

    async def find_completed_candles(self):
        current_time = pd.Timestamp.now()
        active_tokens = False
        
        for token, timeframes in self.data_processor.tick_collector.resampled_buffers.items():
            last_tick_time = self.data_processor.tick_collector.last_tick_time.get(token)
            if last_tick_time is None or (current_time - last_tick_time) > self.IDLE_THRESHOLD:
                self.logger.debug(f"Token {token} idle. Skipping candle end finding.")
                continue

            active_tokens = True

            for timeframe, resampled_data in timeframes.items():
                async with self.data_processor.tick_collector.processing_lock:
                    if not self.data_processor.tick_collector.resampling_enabled[token].get(timeframe, False):
                        continue

                    if not resampled_data:
                        self.logger.debug(f"No resampled data for token {token} and timeframe {timeframe}")
                        continue

                    # Check if there's new data to process
                    last_processed = self.last_processed_candle.get(token, {}).get(timeframe)
                    if last_processed:
                        last_data_time = pd.Timestamp(resampled_data[-1]['tt'])
                        if last_data_time <= pd.Timestamp(last_processed):
                            self.logger.debug(f"No new data for {token} {timeframe} since last processing")
                            continue

                    # Copy data we need to process
                    data_to_process = list(resampled_data)

                # Process data outside the lock
                try:
                    df = pd.DataFrame(data_to_process)
                    df.set_index('tt', inplace=True)
                    
                    freq = pd.Timedelta(timeframe)
                    time_bucket_start = current_time.floor(freq)
                    if len(df) <= 1:
                        self.logger.debug(f"Not enough data for {token} {timeframe}")
                        continue
                    
                    completed_candles = df[df.index < time_bucket_start]

                    if not completed_candles.empty:
                        last_completed_candle = completed_candles.iloc[-1].to_dict()
                        last_completed_candle['tt'] = completed_candles.index[-1].isoformat()

                        # Reacquire lock to update completed_candles_dfs
                        async with self.data_processor.tick_collector.processing_lock:
                            if token not in self.completed_candles_dfs:
                                self.completed_candles_dfs[token] = {}
                            if timeframe not in self.completed_candles_dfs[token]:
                                self.completed_candles_dfs[token][timeframe] = deque(maxlen=self.data_processor.tick_collector.RING_BUFFER_RESAMPLE_SIZE)
                            
                            if (token not in self.last_processed_candle or
                                timeframe not in self.last_processed_candle[token] or
                                self.last_processed_candle[token][timeframe] < last_completed_candle['tt']):
                                
                                self.completed_candles_dfs[token][timeframe].append(last_completed_candle)
                                self.last_processed_candle.setdefault(token, {})[timeframe] = last_completed_candle['tt']
                                
                                self.logger.debug(f"Added new completed candle for {token} {timeframe}: {last_completed_candle}")

                                # Publish the new candle to the queue
                                await self.new_candle_queue.put((token, timeframe, last_completed_candle))  ### pub sub
                   
                    else:
                        self.logger.debug(f"No completed candles for token {token} and timeframe {timeframe}")

                except Exception as e:
                    self.logger.error(f"Error processing token {token} and timeframe {timeframe}: {str(e)}")
                    self.logger.error(f"Traceback: {traceback.format_exc()}")

        if not active_tokens:
            await asyncio.sleep(5)    

    async def get_completed_candles(self, token=None, timeframe=None):
        if token is None and timeframe is None:
            return {t: {tf: list(d) for tf, d in timeframes.items()} 
                    for t, timeframes in self.completed_candles_dfs.items()}
        elif token is not None and timeframe is None:
            return {tf: list(d) for tf, d in self.completed_candles_dfs.get(token, {}).items()}
        elif token is not None and timeframe is not None:
            return list(self.completed_candles_dfs.get(token, {}).get(timeframe, deque()))
        else:  # token is None and timeframe is not None
            return {t: list(timeframes.get(timeframe, deque())) 
                    for t, timeframes in self.completed_candles_dfs.items() if timeframe in timeframes}
        
class TradingDecisionMaker:
    def __init__(self, candle_end_finder: CandleEndFinder):
        self.candle_end_finder = candle_end_finder
        self.logger = logging.getLogger(__name__)
        self.log_directory = "trading_logs"
        os.makedirs(self.log_directory, exist_ok=True)
        
        # Store last 10 candles for each token and timeframe
        self.candle_history = {}
        
        # Control which tokens to trade
        self.active_tokens  = set()
        
        # Control which indicators to use for trading decisions
        self.active_indicators = {
            'supertrend': True,
            'jma': True,
            'candle_average': False
        }
        
        # Control automated trading
        self.automated_trading_enabled = False

    async def run(self):
        while True:
            try:
                token, timeframe, candle = await self.candle_end_finder.new_candle_queue.get()
                await self.process_candle(token, timeframe, candle)
            except Exception as e:
                self.logger.error(f"Error in TradingDecisionMaker: {e}")
                await asyncio.sleep(0.1)

    async def process_candle(self, token, timeframe, candle):
        # Update candle history
        self.update_candle_history(token, timeframe, candle)
        
        # Make trading decision if automated trading is enabled and token is active
        if self.automated_trading_enabled and token in self.active_tokens:
            await self.make_trading_decision(token, timeframe, candle)

    def update_candle_history(self, token, timeframe, candle):
        key = (token, timeframe)
        if key not in self.candle_history:
            self.candle_history[key] = deque(maxlen=10)
        self.candle_history[key].append(candle['close'])

    def get_candle_average(self, token, timeframe):
        key = (token, timeframe)
        if key in self.candle_history and len(self.candle_history[key]) > 0:
            return sum(self.candle_history[key]) / len(self.candle_history[key])
        return None

    def get_log_file_path(self, token, timeframe):
        return os.path.join(self.log_directory, f"{token}_{timeframe}_trading_log.csv")

    def log_to_csv(self, token, timeframe, data):
        file_path = self.get_log_file_path(token, timeframe)
        file_exists = os.path.isfile(file_path)
        
        with open(file_path, mode='a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=data.keys())
            if not file_exists:
                writer.writeheader()
            writer.writerow(data)

    async def make_trading_decision(self, token, timeframe, candle):
        signal = "Neutral"
        reasons = []

        if self.active_indicators['supertrend'] and self.active_indicators['jma']:
            if candle['supertrend_direction'] == 1 and candle['jma_direction'] == 1:
                signal = "Bullish"
                reasons.append("Supertrend and JMA are bullish")
            elif candle['supertrend_direction'] == -1 and candle['jma_direction'] == -1:
                signal = "Bearish"
                reasons.append("Supertrend and JMA are bearish")

        if self.active_indicators['candle_average']:
            avg = self.get_candle_average(token, timeframe)
            if avg is not None:
                if candle['close'] > avg:
                    reasons.append("Price is above 10-candle average")
                    if signal != "Bearish":
                        signal = "Bullish"
                elif candle['close'] < avg:
                    reasons.append("Price is below 10-candle average")
                    if signal != "Bullish":
                        signal = "Bearish"

        self.logger.info(f"Signal for {token} at {timeframe}: {signal}. Reasons: {', '.join(reasons)}")

        # Prepare data for logging
        log_data = {
            'timestamp': datetime.now().isoformat(),
            'timeframe': timeframe,
            'candle_time': candle.get('tt', ''),
            'open': candle.get('open', ''),
            'high': candle.get('high', ''),
            'low': candle.get('low', ''),
            'close': candle.get('close', ''),
            'supertrend': candle.get('supertrend', ''),
            'supertrend_direction': candle.get('supertrend_direction', ''),
            'jma': candle.get('jma', ''),
            'jma_direction': candle.get('jma_direction', ''),
            'candle_average': self.get_candle_average(token, timeframe),
            'signal': signal,
            'reasons': ', '.join(reasons)
        }
        # Log to CSV
        self.log_to_csv(token, timeframe, log_data)

        # Implement your trading logic here based on the signal

    # Methods to control trading settings
    def enable_automated_trading(self):
        self.automated_trading_enabled = True
        self.logger.info("Automated trading enabled")

    def disable_automated_trading(self):
        self.automated_trading_enabled = False
        self.logger.info("Automated trading disabled")

    def add_active_token(self, token):
        self.active_tokens.add(token)
        self.logger.info(f"Added {token} to active trading tokens")

    def remove_active_token(self, token):
        self.active_tokens.discard(token)
        self.logger.info(f"Removed {token} from active trading tokens")

    def enable_indicator(self, indicator):
        if indicator in self.active_indicators:
            self.active_indicators[indicator] = True
            self.logger.info(f"Enabled {indicator} for trading decisions")

    def disable_indicator(self, indicator):
        if indicator in self.active_indicators:
            self.active_indicators[indicator] = False
            self.logger.info(f"Disabled {indicator} for trading decisions")

async def main():
    global global_data_processor
    global global_candle_end_finder, global_tick_collector, global_decision_maker
    collector = TickCollector()
    processor = DataProcessor(collector)
    candle_finder = CandleEndFinder(processor)
    trading_decision_maker = TradingDecisionMaker(candle_finder)
    
    global_tick_collector = collector
    global_data_processor = processor
    global_candle_end_finder = candle_finder
    global_decision_maker = trading_decision_maker
    
    await asyncio.gather(collector.run(), processor.run(), candle_finder.run(),trading_decision_maker.run())

loop = asyncio.get_event_loop()
loop.set_debug(True)
if loop.is_running():
    nest_asyncio.apply()
asyncio.create_task(main())
if not loop.is_running():
    try:
        loop.run_forever()
    except KeyboardInterrupt:
        logger.info("Received exit signal. Cleaning up...")
    finally:
        loop.close()
        logger.info("Event loop closed. Exiting.")

In [ ]:
global_tick_collector.resampled_buffers

In [ ]:
async def get_completed_candles(token=None, timeframe=None):
    global global_candle_end_finder    
    if global_candle_end_finder is None:
        raise ValueError("global_candle_end_finder is not initialized")    
    try:
        completed_candles = await global_candle_end_finder.get_completed_candles(token, timeframe)
        if completed_candles:
            return completed_candles[-1]  # Return the last value
        return None  # Return None if the list is empty
    except Exception as e:
        print(f"Error getting completed candles: {e}")
        return None

# Example usage:
async def example_usage():    
    last_candle = await get_completed_candles(token="432294", timeframe="5s")
    print("Last 1min candle for MCX|432294:", last_candle)

    last_candle = await get_completed_candles(token="432294", timeframe="15s")
    print("Last 5min candle for MCX|432294:", last_candle)

    last_candle = await get_completed_candles(token="26000", timeframe="5s")
    print("Last 1min candle for MCX|432294:", last_candle)

    last_candle = await get_completed_candles(token="26000", timeframe="15s")
    print("Last 5min candle for MCX|432294:", last_candle)

# To run the example:
asyncio.run(example_usage())


In [16]:
# Function to get resampled buffer contents
async def get_buffer_contents(token=None, timeframe=None):
    global global_data_processor
    if global_data_processor is None:
        return "DataProcessor not initialized yet. Please wait a moment and try again."
    return await global_data_processor.get_resampled_buffer_contents(token, timeframe)

# Get all resampled buffer contents
all_contents = await get_buffer_contents()
print("All contents:")
for token, timeframes in all_contents.items():
    print(f"Token {token}:")
    for tf, data in timeframes.items():
        print(f"  Timeframe {tf}: {len(data)} entries")
        if data:
            print(f"    Latest entry: {data[-1]}")  # Show only the latest entry


All contents:
Token 432294:
  Timeframe 5s: 28 entries
    Latest entry: {'tt': Timestamp('2024-10-14 23:00:45'), 'open': 6247.0, 'high': 6247.0, 'low': 6247.0, 'close': 6247.0, 'supertrend': 6244.66, 'supertrend_direction': 1.0, 'jma': 6246.0, 'jma_direction': 1.0}
  Timeframe 15s: 16 entries
    Latest entry: {'tt': Timestamp('2024-10-14 23:00:45'), 'open': 6247.0, 'high': 6247.0, 'low': 6247.0, 'close': 6247.0, 'supertrend': 6243.24, 'supertrend_direction': 1.0, 'jma': 6244.98, 'jma_direction': 1.0}
